In [27]:
import sys
sys.path.append('../')
from models import rnn_classifier ## import 해야 모델 로딩에 오류가 안뜸
from tasks import simple_classification
from fairseq import checkpoint_utils, data, options, tasks
import argparse

In [28]:
parser = options.get_generation_parser(
    default_task='simple_classification',
    )
args = options.parse_args_and_arch(
    parser=parser,
    input_args=[
        '../data/names-bin',
        '--path=../checkpoints/checkpoint_best.pt',
    ]
)
task = tasks.setup_task(args)

| [input] dictionary: 64 types
| [label] dictionary: 24 types


In [29]:
import torch
checkpoint = torch.load(args.path)

In [30]:
print('| loading model from {}'.format(args.path))
models, _model_args = checkpoint_utils.load_model_ensemble([args.path], task=task)
model = models[0]

| loading model from ../checkpoints/checkpoint_best.pt


In [32]:
sentence = 'Satoshi'
print(sentence)


Satoshi


In [36]:
chars = ' '.join(list(sentence.strip()))
chars

'S a t o s h i'

In [37]:
tokens = task.source_dictionary.encode_line(
    chars, add_if_not_exist=False,
)
tokens

tensor([26,  4, 15,  5, 11, 10,  6,  2], dtype=torch.int32)

In [42]:
# Build mini-batch to feed to the model
batch = data.language_pair_dataset.collate(
    samples=[{'id': -1, 'source': tokens}],  # bsz = 1
    pad_idx=task.source_dictionary.pad(),
    eos_idx=task.source_dictionary.eos(),
    left_pad_source=False,
    input_feeding=False,
)

In [43]:
batch

{'id': tensor([-1]),
 'nsentences': 1,
 'ntokens': 8,
 'net_input': {'src_tokens': tensor([[26,  4, 15,  5, 11, 10,  6,  2]], dtype=torch.int32),
  'src_lengths': tensor([8])},
 'target': None}

In [39]:
# Tokenize into characters
chars = ' '.join(list(sentence.strip()))
tokens = task.source_dictionary.encode_line(
    chars, add_if_not_exist=False,
)

# Build mini-batch to feed to the model
batch = data.language_pair_dataset.collate(
    samples=[{'id': -1, 'source': tokens}],  # bsz = 1
    pad_idx=task.source_dictionary.pad(),
    eos_idx=task.source_dictionary.eos(),
    left_pad_source=False,
    input_feeding=False,
)

# Feed batch to the model and get predictions
preds = model(**batch['net_input'])

# Print top 3 predictions and their log-probabilities
top_scores, top_labels = preds[0].topk(k=3)
for score, label_idx in zip(top_scores, top_labels):
    label_name = task.target_dictionary.string([label_idx])
    print('({:.2f})\t{}'.format(score, label_name))

(-0.68)	Japanese
(-0.83)	Arabic
(-3.79)	Polish
